<a href="https://colab.research.google.com/github/alikhitamy/Personal-Projects/blob/master/3CX_Queue_Answered_%2B_Unanswered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Replicating Call Report Answered vs Unanswered Graph

In [ ]:
import pandas as pd

# Read Data from row 5

data_pd=pd.read_csv('callreport(1).csv',header=5)


In [ ]:
# Fill forward the dates for each call

data_pd['Call Time'].fillna(method='ffill',inplace=True)

# Drop the Total Row

data_pd=data_pd.loc[data_pd['Call Time']!='Total:']

# And Blank Row

data_pd=data_pd.loc[data_pd['Call Time']!=""]

In [ ]:
# Split the date + time into seperate date + seperate time for each date-time values
 
for ix in range(0,len(data_pd.index)):
  
  location_of_first_blank_space=data_pd.loc[ix,'Call Time'].find(' ',1)

  date = data_pd.loc[ix,'Call Time'][0:location_of_first_blank_space]

  data_pd.loc[ix,'Call Date Only'] = date

  data_pd.loc[ix,'Call Time Only'] = data_pd.loc[ix,'Call Time'].replace(date,"")

In [ ]:
# Get the hour, minute, second and AM or PM from the time. Then create time groups for counting calls in each time group

for ix in range(0,len(data_pd.index)):
  
  first_colon_position = data_pd.loc[ix,'Call Time Only'].lstrip().find(':',0)

  second_colon_position = data_pd.loc[ix,'Call Time Only'].lstrip().find(':',first_colon_position+1)

  first_blank_space = data_pd.loc[ix,'Call Time Only'].lstrip().find(' ',1)


  hour = int(data_pd.loc[ix,'Call Time Only'].lstrip()[0:first_colon_position])

  minutes = int(data_pd.loc[ix,'Call Time Only'].lstrip()[first_colon_position+1:second_colon_position])

  seconds = int(data_pd.loc[ix,'Call Time Only'].lstrip()[second_colon_position+1:first_blank_space])

  AM_or_PM = data_pd.loc[ix,'Call Time Only'].lstrip()[first_blank_space+1:]

  # 24 Hour Clock -- for working out call durations

  # "{:02d}".format(1) ---> formats 1 as 01 so that there is a leading 0 in front of minutes and seconds

  if AM_or_PM=='PM':

    data_pd.loc[ix,'Call Time Only 24 Hours'] = str(hour+12) + ":" + str("{:02d}".format(minutes)) + ":" + str("{:02d}".format(seconds))
  
  else:
  
    data_pd.loc[ix,'Call Time Only 24 Hours'] = str(hour) + ":" + str("{:02d}".format(minutes)) + ":" + str("{:02d}".format(seconds))
  
  # Create Time Groups

  if hour==12:

    data_pd.loc[ix,'Call Time Group'] = str(hour) + "-" + str(1) + " " + AM_or_PM
  
  else:

    data_pd.loc[ix,'Call Time Group'] = str(hour) + "-" + str(hour + 1) + " " + AM_or_PM

In [ ]:
# Get unique time groups

data_pd['Call Time Group'].unique()

array(['10-11 AM', '11-12 AM', '12-1 PM', '1-2 PM', '2-3 PM', '3-4 PM',
       '4-5 PM', '5-6 PM', '6-7 PM', '7-8 PM', '8-9 PM', '9-10 PM'],
      dtype=object)

# Get Calls by Time Group `Simbisa Queue`

In [ ]:
# Select Simbisa Queue ---> Change this if you want any other queue

data_simbisa_pd = data_pd[data_pd.Destination=='Q Simbisa-Queue (815)']


# For each Call Status, group by Call Time Groups created earlier and count...

for item in data_simbisa_pd.Status.unique():

  calls_by_time_group_pd = (data_simbisa_pd[data_simbisa_pd.Status==item].groupby(['Call Time Group']).count()['Call Time'])

  print(calls_by_time_group_pd)
  
  # Save to CSV
  
  calls_by_time_group_pd.to_csv('Calls by Time Group.csv')

Call Time Group
1-2 PM      487
10-11 AM    164
11-12 AM    259
12-1 PM     442
2-3 PM      371
3-4 PM      406
4-5 PM      499
5-6 PM      693
6-7 PM      653
7-8 PM      462
8-9 PM      201
9-10 PM      38
Name: Call Time, dtype: int64


# Number of calls by agent and duration of calls

In [ ]:
def time_difference_in_hours_minutes_seconds(start_time,end_time):
  
  from datetime import datetime

  time_format = '%H:%M:%S'

  time_delta = datetime.strptime(end_time, time_format) - datetime.strptime(start_time, time_format)

  total_seconds = time_delta.total_seconds()
  
  seconds = total_seconds
  
  seconds = seconds % (24 * 3600) 
    
  hour = seconds // 3600
    
  seconds %= 3600
    
  minutes = seconds // 60
    
  seconds %= 60
      
  return "%d:%02d:%02d" % (hour, minutes, seconds) 

In [ ]:
# Use this to add a new agent each time

agent_counter = 0


# Create a dataframe to save results

number_of_calls_pd = pd.DataFrame(index=[0],
                                  columns=['Agent','Number of Calls','Duration of Calls'])

number_of_calls_pd.index.name='ID'


# For each agent save number of calls


for agent in data_pd[(data_pd.Destination!='Q Simbisa-Queue (815)')&(data_pd.Destination!='IVR (816)')].Destination.unique():

  
  # Add agent
  
  number_of_calls_pd.loc[agent_counter,'Agent'] = agent

  
  # Add Number of Calls

  number_of_calls_pd.loc[agent_counter,'Number of Calls'] = data_pd[data_pd.Destination==agent].shape[0]

  
  # Add Duration of Calls: [0] here is the index of the first call time and [-1] is the index of the last call time

  first_call_time = data_pd[data_pd.Destination==agent].iloc[0]['Call Time Only 24 Hours']
  
  last_call_time = data_pd[data_pd.Destination==agent].iloc[-1]['Call Time Only 24 Hours']

  call_duration = time_difference_in_hours_minutes_seconds(first_call_time,last_call_time)

  number_of_calls_pd.loc[agent_counter,'Duration of Calls'] = call_duration


  # Increment agent to save in dataframe

  agent_counter = agent_counter + 1


# Sort by Number of Calls

number_of_calls_pd.sort_values(by='Number of Calls',ascending=False,inplace=True)

# Save to CSV

number_of_calls_pd.to_csv('Number of Calls and Duration of Calls by Agent.csv')
